In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append('../../')

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import LocalFileStore
from langchain_text_splitters import RecursiveCharacterTextSplitter

from src.ingestion import load_pdf

import os
import uuid

import chromadb
import pickle

In [4]:
RAW_DOCS_PATH = "../../data/raw"
PROCESSED_DOCS_PATH = "../../data/processed"

In [5]:
docs = [load_pdf(os.path.join(RAW_DOCS_PATH, f)) for f in os.listdir(RAW_DOCS_PATH)]

In [23]:
persistent_client = chromadb.PersistentClient(path='../../data/chroma')

In [7]:
def save_to_pickle(obj, filename):
    with open(filename, "wb") as file:
        pickle.dump(obj, file, pickle.HIGHEST_PROTOCOL)

def load_from_pickle(filename):
    with open(filename, "rb") as file:
        return pickle.load(file)

# Parent Child Retriever

In [31]:
def get_parent_child_splits(docs, parent_chunk_size=1200, parent_chunk_overlap=400, child_chunk_size=300, child_chunk_overlap=0, id_key="parent_doc_id"):
    parent_splitter = RecursiveCharacterTextSplitter(
            chunk_size=parent_chunk_size,
            chunk_overlap=parent_chunk_overlap,
            separators=['\n\n\n', '\n\n', '\n', r'\.\s+', ' ', '']
    )

    child_splitter = RecursiveCharacterTextSplitter(
        chunk_size=child_chunk_size,
        chunk_overlap=child_chunk_overlap,
    )

    parent_docs = parent_splitter.split_documents(docs)
    parent_docs_ids = [str(uuid.uuid4()) for _ in parent_docs]

    child_docs = []
    for i, doc in enumerate(parent_docs):
        _id = parent_docs_ids[i]
        _child_docs = child_splitter.split_documents([doc])
        for _doc in _child_docs:
            _doc.metadata[id_key] = _id
        child_docs.extend(_child_docs)

    return parent_docs, parent_docs_ids, child_docs

In [33]:
def get_multivector_retriever(chroma_client, embedding_function, collection_name, save_path, parent_docs=[], parent_docs_ids=[], child_docs=[], id_key="parent_doc_id"):
    
    # Create save directories
    os.makedirs(os.path.join(save_path), exist_ok=True)
    docstore_path = os.path.join(save_path, 'docstore', collection_name)
    vectorstore_path = os.path.join(save_path, 'chroma')

    vectorstore = Chroma(
        client=chroma_client,
        collection_name=collection_name,
        embedding_function=embedding_function,
        persist_directory=vectorstore_path
    )

    store = LocalFileStore(docstore_path)

    # The retriever (empty to start)
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        byte_store=store,
        id_key=id_key,
    )

    # If vectorstore isn't populated, populate and persist
    if not len(retriever.vectorstore.get()['documents']) == len(child_docs):

        if child_docs:
            retriever.vectorstore.add_documents(child_docs)
            retriever.docstore.mset(list(zip(parent_docs_ids, parent_docs)))

            # Save the vectorstore and docstore to disk
            retriever.vectorstore.persist()

    return retriever    

In [17]:
model_id = 'intfloat/multilingual-e5-small'
embedding_function = HuggingFaceEmbeddings(model_name=model_id)

In [29]:
save_path = 'D:\Ahmed\saudi-rag-project\data'

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\lenovo\AppData\Local\Temp\ipykernel_17300\3935730633.py:1: SyntaxWarning: invalid escape sequence '\A'
  save_path = 'D:\Ahmed\saudi-rag-project\data'


In [36]:
%%time

parent_docs, parent_docs_ids, child_docs = get_parent_child_splits(docs)
retriever = get_multivector_retriever(persistent_client, embedding_function, "parent_child_1", save_path, parent_docs, parent_docs_ids, child_docs)

CPU times: total: 15.6 ms
Wall time: 20.9 ms


In [38]:
retriever = get_multivector_retriever(persistent_client, embedding_function, "parent_child_1", save_path)

In [39]:
retriever.invoke("صافي الربح بعد الزكاة")

[Document(page_content='ﺑﻴﺎن أرﺑﺎح اﻟﻨﺘﺎﺋﺞ اﻟﻤﺎﻟﻴﺔ اﻟﺴﻨﻮﻳﺔ ﻟﻠﻌﺎم 2022م\n26 ﻓﺒﺮاﻳﺮ 2023م\nﻧﻈﺮة ﻋﻠﻰ ﻗﺎﺋﻤﺔ اﻟﺮﺑﺢ واﻟﺨﺴﺎرة\nاﻟﻌﺎم اﻟﻤﺎﻟﻲ اﻟﻌﺎم اﻟﻤﺎﻟﻲ اﻟﺘﻐﻴﺮ %\n)ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي(\n2022م 2021م )ﻋﻠﻰ أﺳﺎس ﺳﻨﻮي(\nاﻹﻳﺮادات اﻟﺘﺸﻐﻴﻠﻴﺔ 1071.4 1166.1 %1.8 ▼\nاﻟﻤﺼﺎرﻳﻒ اﻟﺘﺸﻐﻴﻠﻴﺔ 581.1 495.5 %3.71 ▲\nاﻷرﺑﺎح ﻗﺒﻞ اﻟﻔﻮاﺋﺪ واﻟﻀﺮاﺋﺐ واﻻﺳﺘﻬﻼك واﻹﻃﻔﺎء (EBITDA) 490.3 670.6 %9.62 ▼\nﻫﺎﻣﺶ ﻗﺒﻞ اﻟﻔﻮاﺋﺪ واﻟﻀﺮاﺋﺐ واﻻﺳﺘﻬﻼك واﻹﻃﻔﺎء (EBITDA) %64 %85\nﻣﺼﺮوﻓﺎت اﻻﺳﺘﻬﻼك واﻹﻃﻔﺎء 63.2 57.0 %8.01 ▲\nاﻷرﺑﺎح ﻗﺒﻞ اﺣﺘﺴﺎب ﻣﺼﺎرﻳﻒ اﻟﻔﻮاﺋﺪ واﻟﺰﻛﺎة واﻟﻀﺮاﺋﺐ (EBIT) 427.1 613.6 %4.03 ▼\nﻫﺎﻣﺶ اﻷرﺑﺎح ﻗﺒﻞ اﺣﺘﺴﺎب ﻣﺼﺎرﻳﻒ اﻟﻔﻮاﺋﺪ واﻟﺰﻛﺎة واﻟﻀﺮاﺋﺐ (EBIT) %04 %35\nإﻳﺮادات اﻻﺳﺘﺜﻤﺎرات وإﻳﺮادات أﺧﺮى 75.1 43.6 %1.27 ▲\nاﻟﺨﺴﺎرة ﻣﻦ اﻟﺸﺮﻛﺎت اﻟﺰﻣﻴﻠﺔ )9.9( )3.3( %5.202 ▲\nاﻟﺰﻛﺎة 67.7 66.2 %2.2 ▲\nﺻﺎﻓﻲ اﻟﺮﺑﺢ ﺑﻌﺪ اﻟﺰﻛﺎة 424.6 587.7 %8.72 ▼\nﻫﺎﻣﺶ ﺻﺎﻓﻲ اﻟﺮﺑﺢ ﺑﻌﺪ اﻟﺰﻛﺎة %04 %05\nرﺑﺤﻴﺔ اﻟﺴﻬﻢ )رﻳﺎل ﺳﻌﻮدي( 3.54 4.90\nﻧﻈﺮة ﻋﻠﻰ اﻟﻤﺮﻛﺰ اﻟﻤﺎﻟﻲ\nاﻟﻌﺎم اﻟﻤﺎﻟﻲ اﻟﻌﺎم اﻟﻤﺎﻟﻲ اﻟﺘﻐﻴﺮ %\n)ﻣﻠﻴﻮن رﻳﺎل ﺳﻌﻮدي(\n2022م 2021م )ﻋﻠﻰ أﺳﺎس ﺳﻨﻮي(\nاﻟﻨﻘﺪ وﻣﺎ ﻳﻌﺎدﻟﻪ 2118.8 86.2 %1.8532

In [40]:
persistent_client.delete_collection("parent_child_1")